In [1]:
import pandas as pd
import numpy as np
import os, math, json, time, datetime
#from pyspark.sql import SparkSession, functions, window
#from pyspark.sql.types import StringType
#from pyspark.sql.types import StructType
#from pyspark.conf import SparkConf
#from pyspark.sql import functions as func
#from pyspark.sql.window import Window
#from pyspark import SparkContext
#from pyspark import SparkConf,SparkContext,HiveContext
#from pyspark.sql import SQLContext
#from pyspark.sql import SparkSession
#from pyspark.sql import Row
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score, make_scorer

In [3]:
scorecard =pd.read_csv(r'C:\Users\22060909\Documents\Task\WMB\Python\MF2\finalscorecard_tbl.csv')

In [6]:
scorecard = scorecard.drop(['Unnamed: 0'], axis=1)

In [8]:
oot = pd.read_csv(r'C:\Users\22060909\Documents\Task\WMB\Python\MF2\oot_woe_final.csv')

In [41]:
oot_ori = pd.read_csv(r'C:\Users\22060909\Documents\Task\WMB\Python\MF2\OOTFinal.csv')

In [44]:
oot_ori[['CRN','education']].head()

,CRN,education
0,01000041,NaN
1,010000IF,3.0
2,010000RR,5.0
3,010010RA,5.0
4,010010RX,5.0


In [11]:
oot = oot.drop(['Unnamed: 0'], axis=1)

In [17]:
var = list(scorecard['variable'].unique())

In [23]:
var.remove("intercept")

In [46]:
#Scorecard Calculation
oot_score = oot[['CRN','score_swh_bal3']]

#train_score2 = pd.merge(train_copy['education'].astype(str).str[:7], bin_sum_tbl_copy[bin_sum_tbl_copy['variable']=='education'][['woe_7','points']], how='left', left_on='education', right_on ='woe_7', indicator=False)#['points']

for i in var:
    #a = pd.merge(oot[i].astype(str).str[:7].astype(float), scorecard[scorecard['variable']==i][['woe_7','points']], how='left', left_on=i, right_on ='woe_7', indicator=False)#['points']
    a = pd.merge(oot[i], scorecard[scorecard['variable']==i][['woe','points']], how='left', left_on=i, right_on ='woe', indicator=False)#['points']
    a = a.drop(['woe'] + [i], axis = 1)
    a = a.rename(columns = {'points' : i})
    
    a.index = oot.index
    oot_score = pd.merge(oot_score,a, how='left', left_index=True, right_index =True)
oot_score['score_swh_bal_'] = oot_score[var].sum(axis=1)

In [29]:
oot['age'].head()

0    0.061740
1    0.061740
2    0.061740
3    0.030626
4   -0.504671
Name: age, dtype: float64

In [39]:
oot['education'].head()

0   -0.510078
1   -0.454816
2    0.577750
3    0.577750
4    0.577750
Name: education, dtype: float64

In [36]:
oot_score.head(20)

,CRN,score_swh_bal3,growth_pct_Trf_out_Trx_cnt,mob_fin,education,pct_Jen_SA_cnt_growth,max_pastduedays_l19m,marital_status,age,m0_sny_total_aum_cnt,rate_min_mf_of_funding_amt,pct_Jen_TD_cnt_growth,current_funding_trx_amt_over_max,score_swh_bal_
0,01000041,366.542198,72.726582,17.496966,20.50814,56.756690,29.855945,47.812056,41.561156,0.529826,42.892378,24.742142,11.660318,366.542198
1,010000IF,529.417966,72.726582,-11.077669,22.59115,56.756690,99.682304,34.878996,41.561156,84.291339,42.892378,24.742142,60.372899,529.417966
2,010000RR,317.338113,26.968633,-11.077669,61.51188,-81.905917,29.855945,34.878996,41.561156,84.291339,42.892378,24.742142,63.619230,317.338113
3,010010RA,316.417615,26.968633,-11.077669,61.51188,-81.905917,29.855945,34.878996,40.640658,84.291339,42.892378,24.742142,63.619230,316.417615
4,010010RX,297.334800,26.968633,-11.077669,61.51188,-81.905917,29.855945,34.878996,24.804174,84.291339,42.892378,24.742142,60.372899,297.334800
5,010011HM,379.368071,26.968633,-11.077669,61.51188,-81.905917,29.855945,34.878996,41.561156,146.321296,42.892378,24.742142,63.619230,379.368071
6,0100122N,490.410478,72.726582,17.496966,22.59115,56.756690,99.682304,47.812056,41.561156,0.529826,42.892378,24.742142,63.619230,490.410478
7,010012WW,306.930692,26.968633,17.496966,20.50814,56.756690,29.855945,34.878996,40.640658,0.529826,42.892378,24.742142,11.660318,306.930692
8,0100130S,525.648471,26.968633,-11.077669,61.51188,56.756690,29.855945,34.878996,40.640658,84.291339,42.892378,98.556722,60.372899,525.648471
9,01001347,313.514191,26.968633,-11.077669,61.51188,-81.905917,29.855945,47.812056,24.804174,84.291339,42.892378,24.742142,63.619230,313.514191


In [37]:
oot_score.to_csv(r'C:\Users\22060909\Documents\Task\WMB\Python\MF2\oot_score_final.csv')